# Chapter 6 - Interest Rate Futures

## Packages

In [1]:
import datetime as dt
import math
import pandas as pd
import numpy as np
import numpy_financial as npf
import QuantLib as ql

import os
import sys
module_path = os.path.abspath(os.path.join('..\\..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import package.helpers as helpers


## Practice Questions

`6.1`

A U.S. Treasury bond pays a 7% coupon on January 7 and July 7. How much interest accrues per `$`100 of principal to the bondholder between July 7, 2017, and August 8, 2017? How would your answer be different if it were a corporate bond?

In [2]:
print("U.S. Treasury bond")

date1 = dt.date(2017, 7, 7)
date2 = dt.date(2017, 8, 8)
date3 = dt.date(2018, 1, 7)

rate = 0.07 / 2
coupon_rate = rate * 100
print(f"Coupon rate: {round(coupon_rate, 4)}")

num_of_days_between_dates=(date2-date1).days
print(f"Calendar days between {date1} and {date2}: {num_of_days_between_dates} days")

num_of_days_ref_period=(date3-date1).days
print(f"Calendar days between {date1} and {date3}: {num_of_days_ref_period} days")

accrued_interest = coupon_rate * num_of_days_between_dates / num_of_days_ref_period 
print(f"Accrued interest: {round(accrued_interest, 4)}")

helpers.assert_equals(round(accrued_interest, 4), 0.6087)

print("Corporate bond")
print("30/360 day count convention")

num_of_days_between_dates = 31
print(f"Calendar days between {date1} and {date2}: {num_of_days_between_dates} days")

num_of_days_ref_period=180
print(f"Calendar days between {date1} and {date3}: {num_of_days_ref_period} days")

accrued_interest = coupon_rate * num_of_days_between_dates / num_of_days_ref_period 
print(f"Accrued interest: {round(accrued_interest, 4)}")

helpers.assert_equals(round(accrued_interest, 4), 0.6028)


U.S. Treasury bond
Coupon rate: 3.5
Calendar days between 2017-07-07 and 2017-08-08: 32 days
Calendar days between 2017-07-07 and 2018-01-07: 184 days
Accrued interest: 0.6087
Assert okay
Corporate bond
30/360 day count convention
Calendar days between 2017-07-07 and 2017-08-08: 31 days
Calendar days between 2017-07-07 and 2018-01-07: 180 days
Accrued interest: 0.6028
Assert okay


`6.2`

It is January 9, 2018. The price of a Treasury bond with a 6% coupon that matures on October 12, 2030, is quoted as 102-07.

What is the cash price?

In [3]:
date1=dt.date(2017, 10, 12)
current_date=dt.date(2018, 1, 9)
date3=dt.date(2018, 4, 12)

num_of_days_between_dates = (current_date - date1).days
print(f"Number of days between {date1} and {current_date}: {num_of_days_between_dates} days")

num_of_days_ref_period = (date3 - date1).days
print(f"Number of days between {date1} and {date3}: {num_of_days_ref_period} days")

coupon_rate = 0.06/2
print(f"Coupon Rate: {coupon_rate * 100}%")

quoted_price=102+7/32
print(f"Quoted Price: ${round(quoted_price, 2)}")

accrued_interest = num_of_days_between_dates / num_of_days_ref_period * coupon_rate * 100
print(f"Accrued interest: {round(accrued_interest, 4)}")

cash_price = quoted_price + accrued_interest
print(f"Cash Price: ${round(cash_price,2)}")

helpers.assert_equals(round(cash_price,2), 103.69)

Number of days between 2017-10-12 and 2018-01-09: 89 days
Number of days between 2017-10-12 and 2018-04-12: 182 days
Coupon Rate: 3.0%
Quoted Price: $102.22
Accrued interest: 1.467
Cash Price: $103.69
Assert okay


`6.3`

A three-month SOFR futures price changes from 96.76 to 96.82. What is the gain or loss to a trader who is long two contracts?

In [4]:
nb_contracts = 2
print(f"Number of contracts: {nb_contracts}")

price_change = round(96.82 - 96.76, 2)
print(f"Price changes: {price_change}")

gain_per_contract = 100 * 25 * price_change
print(f"Gain per contract: {gain_per_contract}")

total_gain = nb_contracts * gain_per_contract
print(f"Total gain or loss: {total_gain}")

helpers.assert_equals(total_gain, 300.0)

Number of contracts: 2
Price changes: 0.06
Gain per contract: 150.0
Total gain or loss: 300.0
Assert okay


`6.4`

The 350-day LIBOR rate is 3% with continuous compounding and the forward rate calculated from a Eurodollar futures contract that matures in 350 days is 3.2% with continuous compounding. Estimate the 440-day zero rate.

In [5]:
LIBOR_rate = 0.03
print(f"350-day LIBOR rate: {LIBOR_rate * 100} %")

forward_rate = 0.032
print(f"Eurodollar futures forward rate: {forward_rate * 100} %")

forward_maturity = 350
print(f"Eurodollar futures forward maturity: {forward_maturity} days")

zero_period = 440
zero_rate = (forward_rate * 90 + LIBOR_rate * forward_maturity) / zero_period
print(f"Estimated 440-day zero rate: {round(zero_rate * 100, 4)} %")

helpers.assert_equals(round(zero_rate * 100, 4), 3.0409)

350-day LIBOR rate: 3.0 %
Eurodollar futures forward rate: 3.2 %
Eurodollar futures forward maturity: 350 days
Estimated 440-day zero rate: 3.0409 %
Assert okay


`6.5`

It is January 30. You are managing a bond portfolio worth `$`6 million. The duration of the portfolio in 6 months will be 8.2 years. The September Treasury bond futures price is currently 108-15, and the cheapest-to-deliver bond will have a duration of 7.6 years in September. How should you hedge against changes in interest rates over the next 6 months?

In [6]:
quoted_price = 108 + 15/32
print(f"September Treasury bond futures price: {quoted_price}")

portfolio_worth = 6000000
print(f"Portfolio worth: ${portfolio_worth}")

portfolio_duration = 8.2
print(f"Portfolio duration: {portfolio_duration}")

bond_duration = 7.6
print(f"Cheapest-to-deliver bond duration: {bond_duration}")

contract_value = quoted_price * 1000
print(f"Value of a contract: ${contract_value}")

position = "shorted"
print(f"Hedge against changes in interest rates: {position}")

nb_contract = portfolio_worth / contract_value * portfolio_duration / bond_duration
print(f"Number of contracts that should {position}: {round(nb_contract)} contracts")
print("\nThe position should be closed out at the end of July.")

helpers.assert_equals(round(nb_contract), 60)

September Treasury bond futures price: 108.46875
Portfolio worth: $6000000
Portfolio duration: 8.2
Cheapest-to-deliver bond duration: 7.6
Value of a contract: $108468.75
Hedge against changes in interest rates: shorted
Number of contracts that should shorted: 60 contracts

The position should be closed out at the end of July.
Assert okay


`6.6`

The price of a 90-day Treasury bill is quoted as 10.00. What continuously compounded return (on an actual/365 basis) does an investor earn on the Treasury bill for the 90-day period?

In [7]:
duration=90
print(f"{duration}-day Treasury bill")

quoted_price = 10.00
print(f"Quoted price {quoted_price}")

cash_price = 100 - (duration / 360) * quoted_price
print(f"Cash price: ${round(cash_price, 2)}")

cont_return = 365 / duration * math.log(1 + 2.5 / cash_price)
print(f"Annualized continuously compounded return: {round(cont_return * 100, 2)}%")

helpers.assert_equals(round(cont_return, 4), 0.1027)

90-day Treasury bill
Quoted price 10.0
Cash price: $97.5
Annualized continuously compounded return: 10.27%
Assert okay


`6.7`

It is May 5, 2021. The quoted price of a government bond with a 12% coupon that matures on July 27, 2034, is 110-17. What is the cash price?

In [8]:

current_date = dt.date(2021, 5, 5)
print(f"Current Date: {current_date}")

print("Government bond")

quoted_price = 110+17/32
print(f"Quoted price: {round(quoted_price, 4)}")

maturity_date = dt.date(2034, 7, 27)
print(f"Maturity date: {maturity_date}")

coupon = 12
print(f"Coupon: {coupon} %")

date_bgn = dt.date(2021, 1, 27)
print(f"Date Bgn: {date_bgn}")

period1 = (current_date - date_bgn).days
print(f"Number of days between {date_bgn} and {current_date}: {period1} days")

date_end = dt.date(2021, maturity_date.month, maturity_date.day)
print(f"Date End: {date_end}")

period2 = (date_end - date_bgn).days
print(f"Number of days between {date_bgn} and {date_end}: {period2} days")

accrued_interest = (coupon / 2) * period1 / period2
print(f"Accrued interest: {round(accrued_interest, 4)}")

cash_price = quoted_price + accrued_interest
print(f"Cash price: ${round(cash_price, 4)}")

helpers.assert_equals(round(cash_price, 4), 113.7799)

Current Date: 2021-05-05
Government bond
Quoted price: 110.5312
Maturity date: 2034-07-27
Coupon: 12 %
Date Bgn: 2021-01-27
Number of days between 2021-01-27 and 2021-05-05: 98 days
Date End: 2021-07-27
Number of days between 2021-01-27 and 2021-07-27: 181 days
Accrued interest: 3.2486
Cash price: $113.7799
Assert okay


`6.8`

Suppose that the Treasury bond futures price is 101-12. Which of the following four bonds is cheapest to deliver?

Bond | Price  | Conversion factor
---- | ------ | -----------------
1    | 125-05 | 1.2131
2    | 142-15 | 1.3792
3    | 115-31 | 1.1149
4    | 144-02 | 1.4026

In [9]:
print("Treasury bond futures")

futures_price = 101+12/32
print(f"Price: {futures_price} \n")

bonds = {'bond':[1, 2, 3, 4],
         'price':[125+5/32, 142+15/32, 115+31/32, 144+2/32],
         'conversion_factor':[1.2131, 1.3792, 1.1149, 1.4026]}

bonds = pd.DataFrame(bonds)

# The cheapest-to-deliver bond is the one having the lowest factor: Quoted Price - Futures Price * Conversion Factor
bonds = bonds.assign(factor=bonds['price'] - futures_price * bonds['conversion_factor'])

cheapest_bond = bonds['bond'][bonds['factor'].idxmin()]

print(f"Cheapest-to-deliver bond #: {cheapest_bond}")

helpers.assert_equals(cheapest_bond, 4)

Treasury bond futures
Price: 101.375 

Cheapest-to-deliver bond #: 4
Assert okay


`6.9`

It is July 30, 2021. The cheapest-to-deliver bond in a September 2021 Treasury bond futures contract is a 13% coupon bond, and delivery is expected to be made on September 30, 2021. 
Coupon payments on the bond are made on February 4 and August 4 each year. The term structure is flat, and the rate of interest with semiannual compounding is 12% per annum. The conversion factor for the bond is 1.5. The current quoted bond price is \$110. Calculate the quoted futures price for the contract.

In [90]:
date_1 = dt.date(2021, 2, 4)
date_2 = dt.date(2021, 8, 4)
current_date = dt.date(2021, 7, 30)
delivery_date = dt.date(2021, 9, 30)

quoted_price = 110
coupon = 0.13

period1 = (current_date - date_1).days
print(f"Number of days between {date_1} and {current_date} = {period1} days")

period2 = (date_2 - date_1).days
print(f"Number of days between {date_1} and {date_2} = {period2} days")

accrued_interest = (100 * coupon / 2) * period1 / period2
print(f"Accrued interest = {round(accrued_interest, 4)}")

cash_price = quoted_price + accrued_interest
print(f"Cash price = ${round(cash_price, 4)}")

helpers.assert_equals(round(cash_price, 2), 116.32)

interest_rate = 2 * math.log(1.06)
print(f"Interest rate = {round(interest_rate, 4)}")

t = 5/365
print(f"t = {round(t, 4)}")

pv = (100 * coupon / 2) * math.exp(-t * interest_rate)
print(f"Present value = ${round(pv, 4)}")

helpers.assert_equals(round(pv, 4), 6.4896)

future_duration = (delivery_date - current_date).days / 365
print(f"Futures contract lasts for {future_duration} years")


cash_future_price = (cash_price - pv) * math.exp(future_duration * interest_rate)
print(f"Cash futures price for a contract written on the {100 * coupon}% bond = {round(cash_future_price, 2)}")

helpers.assert_equals(round(cash_future_price, 2), 112.03)

n = (delivery_date - date_2).days
print(f"Nb of Days of accrued interest = {n} days")

n2 = 184

conversion_factor = 1.5

quoted_future_price = (cash_future_price - (100 * coupon / 2) * n / n2) / conversion_factor
print(f"Quoted futures price for a contract written on the {100 * coupon}% bond = {round(quoted_future_price, 2)}")

helpers.assert_equals(round(quoted_future_price, 2), 73.34)



Number of days between 2021-02-04 and 2021-07-30 = 176 days
Number of days between 2021-02-04 and 2021-08-04 = 181 days
Accrued interest = 6.3204
Cash price = $116.3204
Assert okay
Interest rate = 0.1165
t = 0.0137
Present value = $6.4896
Assert okay
Futures contract lasts for 0.16986301369863013 years
Cash futures price for a contract written on the 13.0% bond = 112.03
Assert okay
Nb of Days of accrued interest = 57 days
Cash futures price for a contract written on the 13.0% bond = 73.34
Assert okay


`6.10`

A trader is looking for arbitrage opportunities in the Treasury bond futures market. What complications are created by the fact that the party with a short position can choose to deliver any bond with a maturity between 15 and 25 years?

`6.11`

Suppose that the 9-month SOFR interest rate is 8% per annum and the 6-month SOFR interest rate is 7.5% per annum (both with actual/365 and continuous compounding). 

Estimate the 3-month SOFR futures price quote for a contract maturing in 6 months.

`6.12`

Suppose that the 300-day LIBOR zero rate is 4% and Eurodollar quotes for contracts maturing in 300, 398, and 489 days are 95.83, 95.62, and 95.48. Calculate 398-day and 489-day LIBOR zero rates. Assume no difference between forward and futures rates for the purposes of your calculations.

`6.13`

Suppose that a bond portfolio with a duration of 12 years is hedged using a futures contract in which the underlying asset has a duration of 4 years. 

What is likely to be the impact on the hedge of the fact that the 12-year rate is less volatile than the 4-year rate?

`6.14`

Suppose that it is February 20 and a treasurer realizes that on July 17 the company will have to issue \$5 million of commercial paper with a maturity of 180 days. If the paper 
were issued today, the company would realize \$4,820,000. (In other words, the company would receive \$4,820,000 for its paper and have to redeem it at \$5,000,000 in 180 days’ 
time.) The September Eurodollar futures price is quoted as 92.00. 

How should the treasurer hedge the company’s exposure?

`6.15`

On August 1, a portfolio manager has a bond portfolio worth \$10 million. The duration of the portfolio in October will be 7.1 years. The December Treasury bond futures price is currently 91-12 and the cheapest-to-deliver bond will have a duration of 8.8 years at maturity. How should the portfolio manager immunize the portfolio against changes in interest rates over the next 2 months?

`6.16`

How can the portfolio manager change the duration of the portfolio to 3.0 years in Problem 6.15?

`6.17`

Between October 30, 2022, and November 1, 2022, you have a choice between owning a U.S. government bond paying a 12% coupon and a U.S. corporate bond paying a 12% coupon. Consider carefully the day count conventions discussed in this chapter and decide which of the two bonds you would prefer to own. Ignore the risk of default.

`6.18`

The 60-day SOFR rate has been estimated as 3%. The 3-month SOFR futures quote for the period between 60 and 150 days is 96.5. Estimate the 150-day SOFR rate.

`6.19`

Explain why the forward interest rate is less than the corresponding futures interest rate calculated from a Eurodollar futures contract.

`6.20`

It is April 7, 2022. The quoted price of a U.S. government bond with a 6% per annum coupon (paid semiannually) is 120-00. The bond matures on July 27, 2033. What is the cash price? How does your answer change if it is a corporate bond?

`6.21`

It is March 10, 2022. The cheapest-to-deliver bond in a December 2022 Treasury bond futures contract is an 8% coupon bond, and delivery is expected to be made on December 31, 2022. Coupon payments on the bond are made on March 1 and September 1 each year. The rate of interest with continuous compounding is 5% per annum for all maturities. The conversion factor for the bond is 1.2191. The current quoted bond price is \$137. Calculate the quoted futures price for the contract.

`6.22`

Assume that a bank can borrow or lend money at the same interest rate in the LIBOR market. The 90-day rate is 2% per annum, and the 180-day rate is 2.2% per annum, both expressed with continuous compounding and actual/actual day count. The Eurodollar futures price for a contract maturing in 91 days is quoted as 97.95. What arbitrage opportunities are open to the bank?

`6.23`

A Canadian company wishes to create a Canadian LIBOR futures contract from a U.S. Eurodollar futures contract and forward contracts on foreign exchange. Using an example, explain how the company should proceed. For the purposes of this problem, assume that a futures contract is the same as a forward contract.

`6.24`

A portfolio manager plans to use a Treasury bond futures contract to hedge a bond portfolio over the next 3 months. The portfolio is worth `$`100 million and will have a duration of 4.0 years in 3 months. The futures price is 122, and each futures contract is on `$`100,000 of bonds. The bond that is expected to be cheapest to deliver will have a duration of 9.0 years at the maturity of the futures contract. What position in futures contracts is required?

(a) What adjustments to the hedge are necessary if after 1 month the bond that is 
expected to be cheapest to deliver changes to one with a duration of 7 years?

(b) Suppose that all rates increase over the next 3 months, but long-term rates increase 
less than short-term and medium-term rates. What is the effect of this on the 
performance of the hedge?